In [28]:
from multiprocessing import Pool
import matplotlib.pyplot as plt
import pickle
from astropy.coordinates import SkyCoord, match_coordinates_3d, match_coordinates_sky
import sys
import emcee
import numpy as np
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from astropy.table import Table, QTable, hstack, vstack
from astropy import units as u
import corner, time
from scipy.interpolate import interp2d
from astropy.coordinates import SkyCoord, match_coordinates_3d
cosmo_astropy = FlatLambdaCDM(H0=71.0, Om0=0.265, Ob0 = 0.0448)
import iminuit
from iminuit import Minuit
cosmo_astropy.critical_density(0.4).to(u.Msun / u.Mpc**3).value

sys.path.append('/pbs/throng/lsst/users/cpayerne/CLMassDC2/modules/')
import CL_WL_miscentering as mis
import analysis_Mass_Richness_relation as analysis
import CL_WL_two_halo_term as twoh
import CL_WL_mass_conversion as utils
import CL_DATAOPS_match_catalogs as match
import CL_Mass_richness_relation as mr
import CL_Likelihood_for_Mass_richness_relation as likelihood_mr


import clmm
import clmm.dataops
from clmm.dataops import compute_tangential_and_cross_components, make_radial_profile, make_bins
from clmm.galaxycluster import GalaxyCluster
import clmm.utils as u
import astropy.units as un
from clmm import Cosmology
from clmm.support import mock_data as mock
import pyccl as ccl

In [3]:
def save_pickle(dat, filename, **kwargs):
    file = open(filename,'wb')
    pickle.dump(dat, file)
    file.close()

cosmo = Cosmology(H0 = 71.0, Omega_dm0 = 0.265 - 0.0448, Omega_b0 = 0.0448, Omega_k0 = 0.0)
cosmo_astropy = FlatLambdaCDM(H0=71.0, Om0=0.265, Ob0 = 0.0448)
cosmo_clmm = Cosmology(H0 = 71.0, Omega_dm0 = 0.265 - 0.0448, Omega_b0 = 0.0448, Omega_k0 = 0.0)
cosmo_ccl  = ccl.Cosmology(Omega_c=0.265-0.0448, Omega_b=0.0448, h=0.71, A_s=2.1e-9, n_s=0.96, Neff=0, Omega_g=0)

#modeling of the unitary excess surface density profile
def esd_modeling(R, log10m, c, z, cosmo, halo_profile = 'nfw'):
    return clmm.compute_excess_surface_density(R, 10**log10m, c, z, cosmo, delta_mdef=200,
                                       halo_profile_model=halo_profile, massdef='critical')

In [4]:
R = np.linspace(1, 5.5, 10)

In [6]:
logm = np.random.random(100)*(16-13) + 13
c = np.random.random(100)*(20-1) + 1

In [18]:
pos = np.array([[logm[i], c[i]] for i in range(100)])

In [8]:
table = Table()

In [15]:
ds = []
for i in range(100):
    
    ds.append( esd_modeling(R, logm[i], c[i], .3, cosmo, halo_profile = 'einasto') )

In [16]:
from scipy.interpolate import RegularGridInterpolator
interp = RegularGridInterpolator(ds, [logm, c])

ValueError: There are 100 point arrays, but values has 2 dimensions

In [22]:
np.interp([.3, 15],pos, )

TypeError: _interp_dispatcher() missing 1 required positional argument: 'fp'

In [38]:
xa = np.linspace(1, 2, 10)
xb =np.linspace(1, 2, 10)

In [39]:
Xa, Xb = np.meshgrid(xa, xb)

In [44]:
a = np.zeros([100]) + 1

In [47]:
f = interp2d(Xa, Xb, [a, a], kind='cubic')

ValueError: Invalid length for input z for non rectangular grid

In [46]:
f(1, 2)

array([1.])